In [5]:
#pyvizenv environment needed

# Initial imports/modules/libraries
import yfinance as yf
import requests
import os
import pandas as pd
import hvplot.pandas
import panel as pn
import plotly.express as px
import holoviews as hv
import datetime
from dotenv import load_dotenv
import json
import seaborn as sns
from panel.interact import interact
import hvplot.pandas

In [355]:
#tickers_test_list = ["BTC-USD", "DJIA", "GLD", "DX-Y.NYB", "QQQ", "SPY", "WTI"]

def portfolio_builder():
    
    """
    Asks the user how many, and which stocks they'd like to choose for their portfolio.
    """
    
    #Initial Variables
    global list_of_stocks
    list_of_stocks = ["BTC-USD", "DJIA", "GLD", "DX-Y.NYB", "QQQ", "SPY", "WTI"]
    num_stocks='wrong'
    
    #Starts as string, must be turned into "digit".
    #int b4 input would break conditional
    while num_stocks.isdigit() == False:
    
        #Choosing a number
        print(f"Your portfolio currently has {list_of_stocks}.")
        num_stocks = input("How many stocks do you want to add to your portfolio?: ")
        
        #Digit Check
        if num_stocks.isdigit() == False:
            print("Sorry, that's not a digit!")
    
    #A Number has been chosen. Reflect that to the user.
    print(f"We'll add {num_stocks} stocks.\n")

    while (len(list_of_stocks) - 7) < int(num_stocks):
        
        #What stocks are we adding?
        stocks = input("Pick one ticker symbol to add: ")
        
        #Did you put in a number?
        if stocks.isdigit() == True:
            print("Sorry, that's not a valid input!")
            
        #Did you already put that stock in the list?
        elif stocks in list_of_stocks:
            print("You've already chosen that stock. Please pick another!")
            
        #Seems legit. Let's add it to the list!
        else:
            list_of_stocks.append(stocks)
            
    #A list of stocks has been chosen. Reflect that to the user.
    print("------------------------------------------------")
    print(f"Your portfolio will include: ")
    print(*list_of_stocks, sep=", ")
    print("------------------------------------------------")
    pass

In [356]:
portfolio_builder()

Your portfolio currently has ['BTC-USD', 'DJIA', 'GLD', 'DX-Y.NYB', 'QQQ', 'SPY', 'WTI'].
How many stocks do you want to add to your portfolio?: 3
We'll add 3 stocks.

Pick one ticker symbol to add: TSLA
Pick one ticker symbol to add: KO
Pick one ticker symbol to add: GOOG
------------------------------------------------
Your portfolio will include: 
BTC-USD, DJIA, GLD, DX-Y.NYB, QQQ, SPY, WTI, TSLA, KO, GOOG
------------------------------------------------


def volume_grab():
    """
    Takes in arg "ticker" as a str object,
    and returns volume information for that
    symbol/ticker.
    date[index]/open/high/low/close/adj close/volume
    """
    #Initial Lists & variables
    today = pd.to_datetime('today')
    symbol = []
    # For loop grabbing ticker symbol & adding to download function
    for ticker in list_of_stocks:
        symbol.append(ticker)
    #yfinance info
    daily_data = yf.download(symbol,
                            start='2020-01-01',
                            end=today)
    daily_volume = daily_data['Volume']
    
    return daily_volume

In [9]:
def price_grab():
    """
    Takes in arg "ticker" as a str object,
    and returns closing price information for that
    symbol/ticker.
    date[index]/open/high/low/close/adj close/volume
    """
    #Initial Lists & variables
    today = pd.to_datetime('today')
    symbol = []
    # For loop grabbing ticker symbol & adding to download function
    for ticker in list_of_stocks:
        symbol.append(ticker)
    #yfinance info
    daily_data = yf.download(symbol,
                            start='2020-01-01',
                            end=today)
    daily_close = daily_data['Close']
    return daily_close

In [10]:
def pct_change():
    """
    Takes in arg "ticker" as a str object,
    and returns closing price percent change information for that
    symbol/ticker.
    date[index]/open/high/low/close/adj close/volume
    """
    #Initial Lists & variables
    today = pd.to_datetime('today')
    symbol = []
    # For loop grabbing ticker symbol & adding to download function
    for ticker in list_of_stocks:
        symbol.append(ticker)
    #yfinance info
    daily_data = yf.download(symbol,
                            start='2020-01-01',
                            end=today)
    daily_pctchg = daily_data['Close'].pct_change()
    return daily_pctchg

In [11]:
def create_pctchg_panel():
    
    change_graph = pct_change().hvplot(title='Percent Change', 
                                       xlabel='Dates (One Year)',
                                       ylabel='Percent Change',
                                       shared_axes=False,   
                                       width=1000, height=700)
    
    return change_graph

def create_volume_panel():
    
    volume_graph = volume_grab().hvplot(title='Volume', 
                                        xlabel='Dates (One Year)',
                                        yticks=(100000000,200000000,300000000),
                                        shared_axes=False,
                                        ylabel='Volume (Hundred Millions)',
                                        width=1000, height=700)
    
    return volume_graph

def create_price_panel():
    
    price_graph = price_grab().hvplot(title='Price', 
                                      xlabel="Dates (One Year)",
                                      shared_axes=False,
                                      ylabel='Price in USD', 
                                      width=1000, height=700)
    
    return price_graph

def create_heatmap_panel():
    
    correlation = price_grab().pct_change().corr()
    corr_heatmap = sns.heatmap(correlation)
    
    return corr_heatmap

In [12]:
### PANELS NEED WORK, Y LIMITER ON VOLUME GRAPH..
# REFERENCED FUNCTIONS IN GRAPH CREATION - UPDATE? 
### NEEDS REVISING

def create_dashboard():
    """
    Takes the three primary create functions
    and creates HV plot graphs.
    """
    
    # Creating the Graphs for the Panels
    change_graph = create_pctchg_panel()
    volume_graph = create_volume_panel()
    price_graph = create_price_panel()
    
    # Setting up the Panels
    panel_one = pn.Column("##### Price", price_graph)
    panel_two = pn.Column("##### % Change", change_graph)
    panel_three = pn.Column("##### Volume", volume_graph)
    
    # Creating the tabs
    
    dashboard = pn.Tabs(("Prices", panel_one),
                        ("% Change", panel_two),
                        ("Volume",panel_three)
                       )
    
    return dashboard

In [13]:
#NEEDS REVIEW & POSS UPDATES
#Adding and Removing stocks from a portfolio

def add_stock(stock):
    """
    Adds a stock to your portfolio, or list_of_stocks
    """
    list_of_stocks.append(stock)
    print("------------------------------------------------")
    print(f"{stock} was added to your portfolio!")
    print(f"Your portfolio will include: ")
    print(*list_of_stocks, sep=", ")
    print("------------------------------------------------")

def remove_stock(stock):
    """
    Removes a stock from your portfolio, or list_of_stocks
    """
    list_of_stocks.remove(stock)
    print("------------------------------------------------")
    print(f"{stock} was removed from your portfolio!") 
    print(f"Your portfolio will include: ")
    print(*list_of_stocks, sep=", ")
    print("------------------------------------------------")
    pass
    

In [14]:
def rolling_corr():
    """
    Takes in arg "ticker" as a str object,
    and returns daily rolling correlation for that
    symbol/ticker.
    date[index]close.pct_change().rolling(20).corr().dropna()
    """
    #Initial Lists & variables
    today = pd.to_datetime('today')
    symbol = []
    # For loop grabbing ticker symbol & adding to download function
    for ticker in list_of_stocks:
        symbol.append(ticker)
    #yfinance info
    daily_data = yf.download(symbol,
                            start='2020-01-01',
                            end=today)
    daily_close = daily_data['Close'].pct_change().rolling(20).corr().dropna()
    return daily_close

In [322]:
def pick_two():

    #Open variables for later use
    comparison = []

    #Establish Date Range
    today = pd.to_datetime('today')
    start_date = '2020-01-01'

    #Symbol List for data collection
    symbol = []

    # For loop grabbing ticker symbol & adding to download function
    for ticker in list_of_stocks:
        symbol.append(ticker)
    
    #YFinance Data Collection
    daily_data = yf.download(symbol,
                            start=start_date,
                            end=today)

    #Creating a 20-Day rolling correlation & Dropping the first 20 days of NA values.
    rolling_corr = daily_data['Close'].pct_change().rolling(20).corr().dropna()

    #Drop values greater than 0.98
    for stock in list_of_stocks:
        rolling_corr[stock] = rolling_corr[stock][rolling_corr[stock] < 0.98]
    
    #Reset Index to Symbols, dropping dates -- for easier indexing. 
    corr_nodates = rolling_corr.reset_index().drop(columns='Date').set_index('level_1')

    #Choose stocks/indices to compare
    while len(comparison) != 2:
        
        print(f'You can choose from {list_of_stocks}.')
    
        stock1 = input('Please choose the first stock to correlate: ')
    
        if stock1 not in list_of_stocks:
            print(f"Sorry! {stock1} is not in your portfolio! Please choose again.")
        else:
            comparison.append(stock1)
    
        stock2 = input('Please choose your second stock to correlate: ')
    
        if stock2 not in list_of_stocks:
            print(f"Sorry! {stock2} is not in your portfolio! Please choose again.")
        else:
            comparison.append(stock2)

    #Now that we have our two comparison stocks. We can create a correlation graph.

    corr_values = corr_nodates[stock1][stock2].to_frame().reset_index().drop(columns='index')

    return corr_values.hvplot()

In [358]:
pick_two()

[*********************100%***********************]  10 of 10 completed
You can choose from ['BTC-USD', 'DJIA', 'GLD', 'DX-Y.NYB', 'QQQ', 'SPY', 'WTI', 'TSLA', 'KO', 'GOOG'].
Please choose the first stock to correlate: TSLA
Please choose your second stock to correlate: KO


:Curve   [index]   (TSLA)

In [357]:
list_of_stocks

['BTC-USD',
 'DJIA',
 'GLD',
 'DX-Y.NYB',
 'QQQ',
 'SPY',
 'WTI',
 'TSLA',
 'KO',
 'GOOG']